In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


al importarlo note valores nulos y precios fuera de lo normal, los nulos los volvemos 0.0 y los pocos valores erroneos se los acomoda en base a busqueda de los mismos en steam

In [ ]:
ruta = '/content/drive/MyDrive/Proyecto Steam/refined data/recomendacion_juego.csv'
df_recomendacion = pd.read_csv(ruta)
df_recomendacion['price'].fillna(0.0, inplace=True)
df_recomendacion.describe()

,positivo,neutral,negativo,playtime_forever,price,Action,Adventure,Casual,Desconocido,Education,Indie,Massively Multiplayer,RPG,Racing,Simulation,Sports,Strategy,Utilities
count,2928.000000,2928.000000,2928.000000,2.928000e+03,2928.000000,2928.000000,2928.000000,2928.000000,2928.000000,2928.000000,2928.000000,2928.000000,2928.000000,2928.000000,2928.000000,2928.000000,2928.000000,2928.000000
mean,2.238388,15.391052,1.187500,1.269415e+06,11.890956,0.506489,0.315232,0.182377,0.032104,0.002049,0.538593,0.049522,0.200478,0.039959,0.161202,0.034495,0.219262,0.009563
std,13.476670,167.203939,7.435041,1.539000e+07,18.412625,0.500043,0.464688,0.386221,0.176306,0.045229,0.498594,0.216992,0.400427,0.195896,0.367780,0.182527,0.413817,0.097338
min,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,1.000000,0.000000,9.810000e+03,4.990000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,2.000000,0.000000,4.049250e+04,9.990000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,6.000000,1.000000,2.224100e+05,14.990000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,552.000000,8191.000000,323.000000,6.809664e+08,771.710000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
df_sorted = df_recomendacion.sort_values(by='price', ascending=False)
df_sorted

,app_name,positivo,neutral,negativo,playtime_forever,price,Action,Adventure,Casual,Desconocido,Education,Indie,Massively Multiplayer,RPG,Racing,Simulation,Sports,Strategy,Utilities
2856,X-Plane 10 Global - 64 Bit,1,2,0,99906,771.71,0,0,0,0,0,0,0,0,0,1,0,0,0
47,ACE - Arena: Cyber Evolution,0,4,0,23747,189.96,1,0,0,0,0,1,0,0,0,0,1,1,0
1889,RaceRoom Racing Experience,2,14,5,227114,160.91,0,0,0,0,0,0,0,0,1,1,1,0,0
1090,HIS (Heroes In the Sky),0,2,0,17662,139.92,1,0,1,0,0,0,0,1,0,1,0,1,0
1357,Leadwerks Game Engine,0,1,0,120829,99.99,0,0,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1091,HIT,0,1,0,3330,0.00,0,0,0,0,0,0,0,0,0,0,0,0,0
1092,HITMAN™,2,18,0,442819,0.00,1,0,0,0,0,0,0,0,0,0,0,0,0
1106,Half-Life 2: Lost Coast,2,10,1,137607,0.00,1,0,0,0,0,0,0,0,0,0,0,0,0
1107,Half-Life 2: Update,0,4,0,208140,0.00,1,1,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
df_recomendacion.loc[df_recomendacion['app_name'] == 'X-Plane 10 Global - 64 Bit', 'price'] = 19.99
df_recomendacion.loc[df_recomendacion['app_name'] == 'ACE - Arena: Cyber Evolution', 'price'] = 11.89
df_recomendacion.loc[df_recomendacion['app_name'] == 'RaceRoom Racing Experience', 'price'] = 0.00
df_recomendacion.loc[df_recomendacion['app_name'] == 'HIS (Heroes In the Sky)', 'price'] = 0.00


Escalamos las columnas pertinentes para nuestro modelo

In [ ]:
scaler = MinMaxScaler()
df_escalado = df_recomendacion
columnas_a_escalar = ['positivo', 'neutral', 'negativo','playtime_forever']
df_escalado[columnas_a_escalar  ] =  scaler.fit_transform(df_escalado[columnas_a_escalar])

In [ ]:
df_escalado

,app_name,positivo,neutral,negativo,playtime_forever,price,Action,Adventure,Casual,Desconocido,Education,Indie,Massively Multiplayer,RPG,Racing,Simulation,Sports,Strategy,Utilities
0,! That Bastard Is Trying To Steal Our Gold !,0.000000,0.000122,0.000000,0.000001,3.99,1,1,1,0,0,1,0,0,0,0,0,0,0
1,//N.P.P.D. RUSH//- The milk of Ultraviolet,0.001812,0.000122,0.000000,0.000023,3.99,1,0,0,0,0,1,0,0,0,0,0,0,0
2,0RBITALIS,0.000000,0.000122,0.000000,0.000018,9.99,0,0,0,0,0,1,0,0,0,1,0,0,0
3,"10,000,000",0.000000,0.000366,0.000000,0.000189,4.99,1,0,1,0,0,1,0,1,0,0,0,0,0
4,100% Orange Juice,0.000000,0.001221,0.012384,0.000656,6.99,0,0,0,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2923,the static speaks my name,0.000000,0.001587,0.009288,0.000004,0.00,0,1,0,0,0,1,0,1,0,0,0,0,0
2924,theBlu,0.001812,0.000000,0.000000,0.000005,9.99,0,0,0,1,0,0,0,0,0,0,0,0,0
2925,theHunter Classic,0.014493,0.006837,0.043344,0.001542,0.00,1,1,0,0,0,0,0,0,0,1,1,0,0
2926,theHunter: Primal,0.001812,0.001587,0.006192,0.000241,9.99,1,1,0,0,0,0,0,0,0,1,0,0,0


creamos un modelo a partir dee la funcion de similiradidad del coseno, puesto que es un modelo sencillo y facil de implementar

In [ ]:
X = df_escalado[['positivo', 'neutral', 'negativo', 'price', 'Action', 'Adventure', 'Casual', 'Desconocido', 'Education', 'Indie', 'Massively Multiplayer', 'RPG', 'Racing', 'Simulation', 'Sports', 'Strategy', 'Utilities']]
similarity_matrix = cosine_similarity(X)
def obtener_juegos_similares(juego_referencia):
    juego_referencia_index = df_escalado[df_escalado['app_name'] == juego_referencia].index[0]
    similaridades = similarity_matrix[juego_referencia_index]
    juegos_similares_indices = similaridades.argsort()[::-1][1:6]
    juegos_similares = [(df_escalado.iloc[indice]['app_name'], df_escalado.iloc[indice]['price']) for indice in juegos_similares_indices]
    return juegos_similares

In [ ]:
obtener_juegos_similares('0RBITALIS')

[('0RBITALIS', 9.99),
 ('Aerofly FS 1 Flight Simulator', 9.99),
 ('SpaceChem', 9.99),
 ('Hacknet', 9.99),
 ('Poly Bridge', 11.99)]

In [ ]:
df_escalado.to_csv('rec_juego.csv',index = False)